**Caleb and Grady**

Spring 2023

CS 443: Bio-inspired Machine Learning

#### Week 3: Train decoders to classify MNIST from Hebbian network activations

# Project 1: Hebbian Learning

You will use your linear and nonlinear decoders to predict the digit classification accuracy on MNIST based on the learned Hebbian representation (i.e. weights) that you saved last week.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

import time

from mnist import get_mnist
from hebb_net import HebbNet

# plt.style.use(['seaborn-v0_8-colorblind', 'seaborn-v0_8-darkgrid'])
plt.show()
plt.rcParams.update({'font.size': 18})

np.set_printoptions(suppress=True, precision=3)

%load_ext autoreload
%autoreload 2

## Task 5: Train and test decoders on MNIST

### 5a. Preparing decoder inputs

In the cell below:
- load the raw MNIST train/test/validation samples. Reserve 2000 training samples for validation.
- process them with your the Hebbian network (i.e. compute their corresponding netIn values) to get the input for your decoders.
- Cast each of the train/test/validation values to the `tf.float32` datatype.

**Tips:**
- Your Hebbian network constructor has a keyword argument that you can use to load wts from a previously trained network. You should not need to retrain your Hebbian network!
- When creating your Hebbian network object, remember to build it with the same hyperparameters as you did last week (e.g. number of neurons).

In [ ]:
x_train, y_train, x_test, y_test, x_val, y_val = get_mnist(2000)
mnist_data_list = [x_train, x_test, x_val]

net = HebbNet(784, 500, load_wts=True)

mnist_net_in_list = [net.net_in(data_set) for data_set in mnist_data_list]
mnist_net_in_tensors = [tf.convert_to_tensor(net_in, dtype=tf.float32) for net_in in mnist_net_in_list]
print(mnist_net_in_tensors[0].dtype)

### 5b. Linear decoder

Train your softmax classifier on the Hebbian network `net_in` activations obtained from processing the MNIST training set. The hyperparameters are up to you, but you should generally make principled choices (see question below).

#### Guidelines

- Training should be fairly quick (no more than a few minutes).
- Make use of the validation set to monitor progress while training.
- **Remember:** you are **NOT** training the softmax classifier **on MNIST** — you are training it on the `net_in` values produced by the Hebbian network that you trained above!
- Checking the validation accuracy too frequently will slow down training, but if you check too infrequently you won't know what's going on.


#### Results

Here are the items that you should report in the cell(s) below:

- Accuracy of the softmax classifier on the test set.
- Create a well-labeled plot showing the training and validation loss over epochs (*note: because you likely did not check validation loss on each epoch, but "x" epoch values for each loss will likely be different. Account for this when generating "x" values used in the plot.*).

In [ ]:
from neural_decoder import SoftmaxDecoder

In [ ]:
softDecode = SoftmaxDecoder(500, 10)
soft_train_loss_hist, soft_val_loss_hist, n = softDecode.fit(mnist_net_in_tensors[0], y_train, x_val=mnist_net_in_tensors[2], y_val=y_val) 


In [ ]:
plt.figure(figsize = (12,6))
plt.subplot(121)
plt.plot(soft_val_loss_hist, label = "Validation Set Loss")
plt.title("Validation Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")

plt.subplot(122)
plt.title("Training Loss")
plt.plot(soft_train_loss_hist, label = "Training Set Loss", color = 'orange')


plt.xlabel("Epoch")

plt.show()

In [ ]:
pred = softDecode.predict(mnist_net_in_tensors[1])
acc = softDecode.accuracy(y_test,pred)
print(f'Test Set Accuracy: {acc*100}%')

### 5c. Nonlinear decoder

Repeat what you did for the softmax classifier with the nonlinear classifier.

**Note there is one additional step:** Once you get the Hebbian network `net_in` values for the train/validation/test sets, the nonlinear decoding network proposed by Krotov & Hopfield (2019) assumes that the Hebbian network `net_in` values ($h_{ij}$) that serve as the input to the decoder are transformed by the following activation function:

$$x_{ij} = max(h_{ij}, 0)^n$$

where $h_{ij}$ are the Hebbian network `net_in` values. In other words, apply ReLU to the `net_in` values then raise the result to the power `n`. By default, we assume that the hyperparameter $n=5$.

**This additional ReLU step needs to be performed on the `net_in` values representing each of the decoder train/validation/test sets!!**


#### Guidelines

- Training should be fairly quick, but it might take a few minutes longer than the softmax network due to the added complexity.
- The max epochs will need to be set much higher than for the softmax decoder (at least several thousand).
- You likely will need different hyperparameter values here than the softmax network.
- Try using the default $\beta = 0.005$ and $m = 6$ (loss function exponent) hyperparameter values.

In [ ]:
from neural_decoder import NonlinearDecoder

In [ ]:
def preprocess_nonlinear(x, n=5):
    return tf.math.maximum(x,0)**n

In [ ]:
# preprocess and decode
nonlinDecode = NonlinearDecoder(500, 10)
new_x = preprocess_nonlinear(mnist_net_in_tensors[0])
new_val = preprocess_nonlinear(mnist_net_in_tensors[2])
train_loss_hist, val_loss_hist, n = nonlinDecode.fit(x = new_x, y = y_train, x_val=new_val, y_val=y_val, patience = 10)


In [ ]:
# plot
plt.figure(figsize = (13,6))
plt.subplot(121)
plt.plot(val_loss_hist, label = "Validation Set Loss")
plt.title("Validation Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")

plt.subplot(122)
plt.title("Training Loss")
plt.plot(train_loss_hist, label = "Training Set Loss", color = 'orange')


plt.xlabel("Epoch")

plt.subplots_adjust(wspace = 0.5)
plt.show()

In [ ]:
pred = nonlinDecode.predict(preprocess_nonlinear(mnist_net_in_tensors[1]))
acc = nonlinDecode.accuracy(y_test,pred)
print(f'Test Set Accuracy: {acc*100}%')

### 5d. Questions

**Question 9:** Reflect on the relative performance of the linear and nonlinear decoders. In at least one paragraph, compare and contrast pros/cons. 

**Question 10:** Explain your reasoning for picking the hyperparameters that you did for either decoder.

**Question 11:** Which decoder requires more epochs to train until the validation loss/accuracy plateaus? Why do you think that is?

**Answer 9:** At the end of the day, the performance of the linear and nonlinear decoders was pretty similar. Both networks were in the 90's for test set accuracy, though the nonlinear decoder was slightly higher. The main differnce between the networks is the speed in which the training happens. The nonlinear decoder trained faster than the linear decoder even though the nonlinear decoder took more epochs, meaning the linear coder was more computionally intensive. I don't have a particularly beefy computer, so the nonlinear was faster. If you have a high processing power computer, and you are willing to sacrifice some accuracy for a quicker training time, perhaps the linear decoder is the better option. While there were a few differences in the implementation of the two networks, neither one was way more difficult to code than the other. The only other major difference was the was the extra preprocessing step for the nonlinear decoder, but that was only required once and wasn't a problem at all. In the end, both networks are incredibly similar and it really comes down to one thing. Do you want to prioritize fewer slower epochs or more faster epochs?

**Answer 10:** We were getting pretty good performance on both networks with the base hyperparameters. On the advice of Professor Layton, we increased the accuracy of the nonlinear decoder to 10, which resulted in ~5% bump in accuracy. However, this produce a validation loss graph that is trending slightly upward, so perhaps a slightly lower patience value could further increase performance. The linear decoder's validation loss is very erratic, so perhaps increasing the batch size could further increase its performance. 

**Answer 11:** The nonlinear decoder requires significantly more iterations until the losses plateau. This might be because values falling outside of the steep region of the tanh function. This would cause different input samples to have similar outputs, preventing the network from quickly differentiating between samples.

## Extensions

### 0. Compare encoder-decoder model to end-to-end training

Compare how accurately the linear and nonlinear decoders learn from raw MNIST samples compared to from the Hebbian network activations (*I would suggest keeping hyperparameters constant for a fair comparison*). There is a lot to explore here! Here are a few questions to examine:
- How rapidly the decoders networks learn their inputs (e.g. number of training epochs needed to achieve "good" accuracy on the validation set)?
- What is the best test accuracy achieved by the decoders networks with the same hyperparameters?
- How does the decoder training times compare with the Hebbian inputs vs raw MNIST samples to achieve some level of accuracy?
- Remember that you control the dimension of the "embedding" performed by the Hebbian network (i.e. number of neurons in the net). How does the accuracy and/or training time of the decoders trade off with the Hebbian network embedding size?

### 1. Use your CS 343 Softmax network as the linear decoder

This will require a few updates to support the Adam optimizer (that you implemented in the CS 343 CNN project) and validation sets.

Copy `softmax_layer.py` from your CS343 MLP project to your working directory. Also copy `optimizer.py` from your CS343 CNN project.

Make the following changes to `fit()` in `softmax_layer.py`:
1. Switch your optimizer from SGD to Adam. This will involve creating two `Adam` objects: one for the weights, one for the bias. Also, be sure to set the Adam learning rate based on the value passed into `fit()`.
2. Add support in `fit()` for a validation set by adding the keyword arguments: `x_val=None, y_val=None`. If `verbose > 0` print out the accuracy and loss over the entire validation set. 
3. If `verbose > 0` convert your print outs to happen in terms of epochs rather than iterations (e.g. every epoch, not every 100 iterations). Add a keyword argument `val_freq=50` to specify how often (in epochs) to check and print out the validation accuracy and loss. Be sure to always print out the validation accuracy and loss on the first and last epoch regardless of the `val_freq` value.
4. Have `fit()` return both the train and validation loss as Python lists or ndarrays. In cases when you do not pass in a validation set, the returned validation loss list may be `None` and that's ok.

The network should train similarily to your Tensorflow version. Compare/analyze runtime performance.

### 2. Encode an image dataset of your choice with the Hebbian network

For example, Fashion MNIST, STL-10 or CIFAR-10. If your images contain color, I suggest either converting to grayscale or flattening the color channels when constructing your feature vectors (e.g. `(32, 32, 3)` color image made into a `(3072,)` vector). Note that color images will clearly take much longer to train.

Some areas to explore:
- Visualize the weights. Analyze how hyperparameters affect the structure.
- Compare decoding accuracy


### 3. Learning rate decay

Krotov & Hopfield (2019) decayed the learning rate according to a epoch-based schedule for both the Hebbian network and the decoder (see their Appendix B for details). Implement this or your own variant (for either encoder and/or decoder network) and explore whether it improves decoding performance.

### 4. Hyperparameter tuning

Use a grid or random search for encoder and/decoder networks to optimize performance.

How does the number of "runner-up" neurons in the Hebbian network ($K$) influence the learned weights (visually) or the accuracy with which either decoding network decodes the correct digit?

### 5. Visualize Hebbian network activations

- Develop a way to visualize and gain a better intuition about how a Hebbian neuron's learned weights contribute to the activation across the network. One idea is to normalize each neuron's learned weights between [0, 1] (or in a way that preserves negative wts) after training. Then to visualize the contributions for a given sample `i`, scale each neuron's weights by the its netIn value to sample `i`. Draw the scaled weights using `draw_grid_image`. It might be helpful to do this for a Hebbian network with a smaller number of neurons so that you can legibly plot every neuron's weights in one plot. Analyze/interpret the relationship between this weight plot and the decoded predictions.
- Figure out how to record the "live" Hebbian training weight plots into videos for later viewing.

This are only two ideas — try out other ideas that come to mind!

### 6. Confusion matrix and error analysis of MNIST classification

For one or both classifier, make a confusion matrix of the digit classifications. Use your confusion matrix to gain insight into misclassifications. Run follow-up analyses/training sessions to explore patterns in more depth. For example, if two classes are frequently misclassified, how neurons in the Hebbian network develop receptive fields that resemble each? Are the weights resembling the two classes strongly correlated (and how?)? To what degree are inhibitory weights learned for these neurons? What happens if you train the Hebbian network on only samples belonging to the two classes — do classes of either class become less/more confusable? And so forth...

### 7. Hebbian network in TensorFlow

Implement the Hebbian network using TensorFlow rather than Numpy. Quantify/compare the runtime performance of each version.

Notes: 
- Run your analysis for different mini-batch sizes. Start small, but try a wide range of sizes make sure your computer can handle the values you select. The TensorFlow version may only be faster for certain mini-batch sizes.
- You probably will notice the biggest difference if you have GPU accelerated TensorFlow working on your computer.

### 8. Implement the Generalized Hebbian Algorithm (GHA) and compare to PCA

The GHA provides an incremental version of PCA — compute PCA one sample at a time over a number of training epochs. This approach can be helpful when you want to run PCA on a large dataset, but the dataset is too large to fit in your computer's memory (e.g. perhaps STL-10 at full 96x96 resolution). 

Implement GHA then show for a large dataset (e.g. STL-10) that GHA computes the PCA representation, whereas regular PCA (e.g. from CS251/2) fails. Plot what the image samples look like over training epochs when projected to PCA space and then back to the original data space (i.e. filtered by the learned principle components / network weights). If this sounds interesting, please see me for guidance.

### 9. Experiment with different decoder architectures

Create one or more different nonlinear decoders in TensorFlow (e.g. MLP, CNN). Compare performance/accuracy with the nonlinear one in the project.

### 10. Tuning Hebbian network

Experiment with how the Hebbian network hyperparameters (number of neurons, which neuron gets inhibited, inhibition strength, etc.) affect encoding of the MNIST digits and decoding accuracy.

In [ ]:
#Extension: Affect of nonlinear preprocessing on linear network
ext_x = preprocess_nonlinear(mnist_net_in_tensors[0])
ext_val = preprocess_nonlinear(mnist_net_in_tensors[2])

extNet = SoftmaxDecoder(500, 10)
ext_train_loss_hist, ext_val_loss_hist, n = extNet.fit(ext_x, y_train, 
                                                       x_val=ext_val, y_val=y_val) 



In [ ]:
# plot
plt.figure(figsize = (13,6))
plt.subplot(121)
plt.plot(ext_val_loss_hist, label = "Validation Set Loss")
plt.title("Validation Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")

plt.subplot(122)
plt.title("Training Loss")
plt.plot(ext_train_loss_hist, label = "Training Set Loss", color = 'orange')


plt.xlabel("Epoch")

plt.subplots_adjust(wspace = 0.5)
plt.show()

In [ ]:
pred = extNet.predict(preprocess_nonlinear(mnist_net_in_tensors[1]))
acc = extNet.accuracy(y_test,pred)
print(f'Test Set Accuracy: {acc*100}%')

The above uses a linear decoder and has identical hyperparameters to the network in 5b. However, this network uses the extra preprocessing step from 5c and applies the $ReLu^n$ function to the data before giving it to the network. I used the defualt value of $n=5$ for the first test. The result is an immediate jump to 96% accuracy on the test set, a 4% increases over 5b. Additionally, it didn't seem to affect the training time or loss curves. 

The $ReLu^n$ function significantly increases differentiation between positive values, but decreases differentiation between negative values. Obviously, the changes to the dataset help the network more than they hurt it. There were more positive weights than negative weights in the encoder net, so there are probably more positive values in the resulting encoded dataset. This would cause the benefit of increased differtiation on positive values to out-weigh to cons of setting all negative values to 0, since there just aren't that many negative values. The next question is if there is a value of $n$ that increases performance even more.

In [ ]:
#Extension: IDK Bro its some funky shit
ext_x2 = preprocess_nonlinear(mnist_net_in_tensors[0], 5)
ext_val2 = preprocess_nonlinear(mnist_net_in_tensors[2], 5)

extNet2 = SoftmaxDecoder(500, 7)
ext_train_loss_hist2, ext_val_loss_hist2, n2 = extNet2.fit(ext_x2, y_train, 
                                                       x_val=ext_val2, y_val=y_val) 

In [ ]:
pred2 = extNet2.predict(preprocess_nonlinear(mnist_net_in_tensors[1], 5))
acc = extNet2.accuracy(y_test,pred2)
print(f'Test Set Accuracy: {acc*100}%')

We used the above code to test several different values of $n$. When $n=10$, the net had an accuracy of 94% for some iteratins, which is better than the base net but worse thatn $n=5$, but this was not also consistent. We also tried a value of $n=1$, producing an accuracy of 64%. This is much worse than the original network. The network achieved a similar, low accuracy with $n=7$. It seems as though the effect of the nonlinear preprocessing is general unpredictable. However, some values of $n$ benefit the network, so through grid search would be necessary to find the value of $n$ that benefits the network the most. It's possible that we stumbled on one of the best values for our softmax decoder. One other caveat is that the effect was not always consistent, and we sometimes observed low accuracies for values of $n$ that had worked well previously. We have included a screenshot of a high accuracy test with the zip file. However, we are unsure why the inconsistence exists.